In [34]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [35]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [36]:
# 1. Read in the AWS S3 bucket into a DataFrame.
import requests
from pyspark import SparkFiles

# Download the file from the URL
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
local_path = "home_sales_revised.csv"

response = requests.get(url)
with open(local_path, 'wb') as file:
    file.write(response.content)

# Now read the file into a Spark DataFrame
df = spark.read.csv(local_path, header=True, inferSchema=True)

In [37]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

In [38]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- date_built: integer (nullable = true)
 |-- price: integer (nullable = true)
 |-- bedrooms: integer (nullable = true)
 |-- bathrooms: integer (nullable = true)
 |-- sqft_living: integer (nullable = true)
 |-- sqft_lot: integer (nullable = true)
 |-- floors: integer (nullable = true)
 |-- waterfront: integer (nullable = true)
 |-- view: integer (nullable = true)



In [39]:
# 3. What is the average price for a four bedroom house sold per year, rounded to two decimal places?
from pyspark.sql.functions import year, avg, round

average_price_df = (
    df.filter(df.bedrooms == 4)
    .withColumn("year", year(df.date))  # Use 'date' instead of 'sale_date'
    .groupBy("year")
    .agg(round(avg("price"), 2).alias("average_price"))
    .orderBy("year")
)

average_price_df.show()

+----+-------------+
|year|average_price|
+----+-------------+
|2019|     300263.7|
|2020|    298353.78|
|2021|    301819.44|
|2022|    296363.88|
+----+-------------+



In [40]:
# 4. What is the average price of a home for each year the home was built,
# that have 3 bedrooms and 3 bathrooms, rounded to two decimal places?
from pyspark.sql.functions import avg, round

filtered_df = df.filter((df.bedrooms == 3) & (df.bathrooms == 3))

average_price_df = (
    filtered_df
    .groupBy("date_built")  # Use 'date_built' instead of 'year_built'
    .agg(round(avg("price"), 2).alias("average_price"))
    .orderBy("date_built")
)
#show reference
average_price_df.show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [41]:
# 5. What is the average price of a home for each year the home was built,
# that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet, rounded to two decimal places?
from pyspark.sql.functions import year, avg, round

# Filter for homes with 3 bedrooms and 3 bathrooms
filtered_df = df.filter((df.bedrooms == 3) & (df.bathrooms == 3))

# Extract the year from date_built and calculate the average price
average_price_df = (
    filtered_df
    .withColumn("year_built", df.date_built)  # Extract the year from 'date_built'
    .groupBy("year_built")
    .agg(round(avg("price"), 2).alias("average_price"))
    .orderBy("year_built")
)

# Show the results
average_price_df.show()

+----------+-------------+
|year_built|average_price|
+----------+-------------+
|      2010|    292859.62|
|      2011|    291117.47|
|      2012|    293683.19|
|      2013|    295962.27|
|      2014|    290852.27|
|      2015|     288770.3|
|      2016|    290555.07|
|      2017|    292676.79|
+----------+-------------+



In [42]:
# 6. What is the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000? Order by descending view rating. 
# Although this is a small dataset, determine the run time for this query.
import time
from pyspark.sql.functions import round, avg

start_time = time.time()

average_price_per_view_df = (
    df.groupBy("view")
    .agg(round(avg("price"), 2).alias("average_price"))
)

filtered_df = average_price_per_view_df.filter(average_price_per_view_df.average_price >= 350000)

result_df = filtered_df.orderBy("view", ascending=False)

result_df.show()

end_time = time.time()

print("--- %s seconds ---" % (end_time - start_time))

+----+-------------+
|view|average_price|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.2870643138885498 seconds ---


In [43]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("home_sales")

In [44]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [45]:
# 9. Using the cached data, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the uncached runtime.
from pyspark.sql.functions import avg, round
import time

# 1. Run the query without caching and measure the time
start_time = time.time()

uncached_result = (
    df.groupBy("view")
    .agg(round(avg("price"), 2).alias("average_price"))
    .filter("average_price >= 350000")
    .orderBy("view")
)

uncached_result.show()

uncached_runtime = time.time() - start_time
print(f"Uncached runtime: {uncached_runtime:.2f} seconds")

# 2. Cache the DataFrame or table if not already cached
spark.catalog.cacheTable("home_sales")

# 3. Run the query using cached data and measure the time
start_time = time.time()

cached_result = (
    df.groupBy("view")
    .agg(round(avg("price"), 2).alias("average_price"))
    .filter("average_price >= 350000")
    .orderBy("view")
)

cached_result.show()

cached_runtime = time.time() - start_time
print(f"Cached runtime: {cached_runtime:.2f} seconds")

# 4. Compare the runtimes
print(f"Performance improvement: {uncached_runtime / cached_runtime:.2f}x faster")

+----+-------------+
|view|average_price|
+----+-------------+
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44|
|  69|    750537.94|
|  70|    695865.58|
+----+-------------+
only showing top 20 rows

Uncached runtime: 0.41 seconds
+----+-------------+
|view|average_price|
+----+-------------+
|  51|    788128.21|
|  52|    733780.26|
|  53|     755214.8|
|  54|    798684.82|
|  55|    771153.32|
|  56|     718176.4|
|  57|     734340.5|
|  58|    759764.65|
|  59|     791453.0|
|  60|    754939.65|
|  61|    746877.59|
|  62|    759150.14|
|  63|    711614.55|
|  64|    767036.67|
|  65|    736679.93|
|  66|     712475.0|
|  67|    737970.96|
|  68|    716785.44

In [46]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Set HADOOP_HOME environment variable
os.environ["HADOOP_HOME"] = "C:/Users/higas/Downloads/hadoop-3.4.0"

# Initialize SparkSession
spark = SparkSession.builder.appName("PartitionParquet").getOrCreate()

# Assuming `df` is your DataFrame
df = df.withColumn("date_built", to_date(col("date_built").cast("string"), "yyyy-MM-dd"))

# Define the output path
output_path = "C:/Users/higas/Downloads/Future Assignments Bootcamp/Module 22 Challenge/Starter_Code/partitioned_data"

# Write DataFrame to Parquet with partitioning and overwrite mode
df.write.mode("overwrite").partitionBy("date_built").parquet(output_path)

print("Data written to Parquet with partitioning by 'date_built'.")



Py4JJavaError: An error occurred while calling o414.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: Hadoop home directory C:\Users\higas\Downloads\hadoop-3.4.0 does not exist -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSy

In [ ]:
# 11. Read the formatted parquet data.
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadParquet").getOrCreate()

# Adjust this path based on where you actually saved the Parquet files
parquet_path = "C:\Users\higas\Downloads\Future Assignments Bootcamp\Module 22 Challenge\Starter_Code\home_sales_revised.cs"

# Read the Parquet data
df_parquet = spark.read.parquet(parquet_path)

df_parquet.printSchema()
df_parquet.show()


In [ ]:
# 12. Create a temporary table for the parquet data.
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ReadParquet").getOrCreate()

parquet_path = "path/to/save/home_sales_partitioned"

df_parquet = spark.read.parquet(parquet_path)

df_parquet.createOrReplaceTempView("home_sales_parquet")

In [ ]:
# 13. Using the parquet DataFrame, run the last query above, that calculates 
# the average price of a home per "view" rating, rounded to two decimal places,
# having an average home price greater than or equal to $350,000. 
# Determine the runtime and compare it to the cached runtime.
import time
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

# Define the parquet path
parquet_path = "C:/Users/higas/Downloads/Future Assignments Bootcamp/Module 22 Challenge/Starter_Code/partitioned_data"

# Read the parquet data
df_parquet = spark.read.parquet(parquet_path)

# Register DataFrame as a temporary view
df_parquet.createOrReplaceTempView("home_sales_parquet")

# Measure runtime for parquet data
start_time_parquet = time.time()

average_price_per_view_df_parquet = spark.sql("""
    SELECT view, ROUND(AVG(price), 2) AS average_price
    FROM home_sales_parquet
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view DESC
""")

average_price_per_view_df_parquet.show()

end_time_parquet = time.time()

# Print the runtime
print("--- Parquet Data Runtime: %s seconds ---" % (end_time_parquet - start_time_parquet))


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached
is_cached = spark.catalog.isCached("home_sales")